# Proyecto final. Natural Languaje Processing
### Demo. Modelo BERT para finetuning

Guillermo Segura Gómez

#### ¿Qué es un modelo BERT?

**BERT (Bidirectional Encoder Representations from Transformers)** es un modelo de lenguaje desarrollado por Google. Es uno de los modelos más avanzados en procesamiento de lenguaje natural (NLP). BERT es un transformador bidireccional, lo que significa que tiene en cuenta el contexto de las palabras tanto a la izquierda como a la derecha de una palabra en una oración.

##### Características Clave de BERT:
1. **Bidireccionalidad**: Analiza el contexto de una palabra en ambos sentidos (izquierda y derecha).
2. **Pre-entrenamiento y Fine-tuning**: BERT se pre-entrena en grandes cantidades de texto sin etiquetar (como Wikipedia) y luego se puede ajustar finamente (fine-tuning) para tareas específicas.
3. **Transformadores**: Utiliza la arquitectura de transformadores, que es altamente eficiente para capturar dependencias a largo plazo en texto.

##### Aplicaciones Comunes de BERT:
1. **Clasificación de Texto**: Clasificación de sentimientos, clasificación de spam, etc.
2. **Reconocimiento de Entidades Nombradas (NER)**: Identificación de nombres de personas, organizaciones, ubicaciones, etc.
3. **Respuesta a Preguntas**: Sistemas de preguntas y respuestas donde el modelo encuentra la respuesta en un párrafo de texto.
4. **Traducción de Idiomas**: Traducción automática de texto de un idioma a otro.

In [4]:
# Librerias
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
import torch
import json
import argparse
import os
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import random
import numpy as np
from datetime import datetime
from tqdm import tqdm
import sys

# Configuración del dispositivo (CPU o GPU)
if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"
print(f"Using device={DEVICE}")

# Función para mover los datos a la GPU si está disponible
def to_cuda(var):
    if DEVICE == "cuda":
        return var.cuda()
    return var

# Definición de métricas de evaluación
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, zero_division=0.0, average="binary", pos_label=1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tp': tp,
        'tn': tn,
        'fp': fp,
        'fn': fn
    }

Using device=cpu


#### 1. **Arquitectura de BERT**

BERT consta de una serie de capas de atención (self atention layers) que procesan el texto de entrada para producir representaciones contextuales de cada token.

##### Componentes Principales de BERT:

1. **Tokenización**: BERT utiliza un tokenizador para dividir el texto en tokens (subpalabras) y agregar tokens especiales como `[CLS]` (inicio de secuencia) y `[SEP]` (separador de secuencias).
2. **Embeddings**: Los tokens son convertidos a vectores de embeddings que capturan información semántica.
3. **Capas de Transformadores**: Una serie de capas de atención bidireccional que procesan los embeddings para generar representaciones contextuales de cada token.
4. **Salida**: La salida de la última capa de BERT es una secuencia de vectores que representan cada token en su contexto.

#### 2. **Proceso de Fine-Tuning**

El fine-tuning es el proceso de ajustar un modelo preentrenado a una tarea específica utilizando datos etiquetados. En este caso clasificación de texto, esto implica ajustar los pesos del modelo utilizando un conjunto de datos donde cada texto tiene una etiqueta correspondiente.

##### Componentes del Fine-Tuning en BERT:

1. **Capa de Clasificación**: Se agrega una capa de clasificación al final del modelo BERT. Esta capa toma la representación del token `[CLS]` y la usa para predecir la etiqueta de la secuencia de entrada.
2. **Entrenamiento**: El modelo se entrena utilizando los datos etiquetados. Durante el entrenamiento, se ajustan todos los pesos del modelo BERT (o solo algunos, dependiendo de la configuración).

#### 3. **Etiquetas y Salidas en BERT**

##### 3.1 Etiquetas en Datos de Entrenamiento

Las etiquetas en los datos de entrenamiento son las clasificaciones asignadas a cada secuencia de entrada. 

##### 3.2 Salidas del Modelo

La salida de BERT para una tarea de clasificación es un vector de logits que se convierte en una probabilidad para cada clase mediante una función softmax. La clase con la mayor probabilidad se selecciona como la predicción del modelo.

In [5]:
# Función para manejar los datos de entrada
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# Función para simulación de datos mal etiquetados
def mislabel_data(data, percent):
    print(f"WARNING: Mislabeling {percent}% of data ({int(len(data)*(percent/100))} lines)!")
    indices_to_change = random.sample(range(len(data)), int(len(data)*(percent/100)))
    for idx in indices_to_change:
        if data[idx]["label"] == 0:
            data[idx]["label"] = 1  
        else:
            data[idx]["label"] = 0


In [ ]:
# Función para cargar los datos de un archivo y tokenizarlos con el tokenizador de BERT
def load_dataset_from_file(fpath, tokenizer, cfg):
    with open(fpath, "r") as file:
        data = json.load(file)
    if cfg["reduce_lines_for_testing"]:
        data = data[:100]
    encodings = tokenizer([d["txt"] for d in data], truncation=True, padding=True, max_length=cfg["max_len"])
    return data, SentimentDataset(encodings=encodings, labels=[d["label"] for d in data])


In [11]:
# Función principal
def main(cfg):
    # Definición de parámetros por defecto
    default_params = {
        "num_epochs": 5,
        "batch_size": 16,
        "max_len": 512,
        "model_save_location": None,
        "freeze_encoder": False,
        "log_test_outputs": False,
        "train_fpath": None,
        "dev_fpath": None,
        "test_fpath": None,
        "inference_fpath": None,
        "inference_results_savepath": None,
        "final_test_fpath": None,
    }
    # Sobrescribir los parámetros por defecto con los valores del archivo de configuración
    for key in default_params:
        cfg[key] = cfg.get(key, default_params[key])

    # Cargar el tokenizer y el modelo preentrenado de BERT
    tokenizer = AutoTokenizer.from_pretrained(cfg["model_name"])
    model = to_cuda(AutoModelForSequenceClassification.from_pretrained(cfg["model_name"], num_labels=2))


    # Cargar los datos de entrenamiento, validación y prueba
    if cfg["train_fpath"] is not None:
        # Advertencia si se está reduciendo el tamaño de los datos para pruebas
        if cfg["reduce_lines_for_testing"]:
            print("WARNING: Keeping only 100 sentences for test and train for testing!")

        # Cargar y tokenizar los datos de entrenamiento
        train_data, train_dataset = load_dataset_from_file(cfg["train_fpath"], tokenizer, cfg)

        eval_datasets = {}

        # Cargar y tokenizar los datos de validación si están disponibles
        if cfg["dev_fpath"] is not None:
            dev_data, eval_datasets["dev"] = load_dataset_from_file(cfg["dev_fpath"], tokenizer, cfg)
        
        # Cargar y tokenizar los datos de prueba si están disponibles
        if cfg["test_fpath"] is not None:
            test_data, eval_datasets["test"] = load_dataset_from_file(cfg["test_fpath"], tokenizer, cfg)

        final_test_dataset = None
        # Asignar los datos de prueba finales
        if cfg["final_test_fpath"] is None and cfg["test_fpath"] is not None:
            final_test_data = test_data
            final_test_dataset = eval_datasets["test"]
        elif cfg["final_test_fpath"] is not None:
            final_test_data, final_test_dataset = load_dataset_from_file(cfg["final_test_fpath"], tokenizer, cfg)

        # Simular datos mal etiquetados si se especifica
        if int(cfg["mislabel_percent"]) != 0:
            mislabel_data(train_data, cfg["mislabel_percent"])

        # Imprimir el número de muestras cargadas para cada conjunto de datos
        print(f"Number of train samples loaded: {len(train_data)}")
        if cfg["dev_fpath"] is not None:
            print(f"Number of dev samples loaded: {len(eval_datasets['dev'])}")
        if cfg["test_fpath"] is not None:
            print(f"Number of test samples loaded: {len(eval_datasets['test'])}")
        if cfg["final_test_fpath"] is not None:
            print(f"Number of final test samples loaded: {len(final_test_dataset)}")

        # Configuración estrategía de evaluación y congelación de parámetros del modelo

            # Definir la estrategia de evaluación
        if cfg["dev_fpath"] or cfg["test_fpath"]:
            evaluation_strategy = "steps"
        else:
            evaluation_strategy = "no"
        print(f"Using evaluation strategy {evaluation_strategy}")

        # Congelar los parámetros del encoder si se especifica
        if cfg["freeze_encoder"]:
            for name, param in model.named_parameters():
                if "embedding" in name or "encoder" in name:
                    param.requires_grad = False

        # Configuración de argumentos del entrenamiento

            # Configuración de los pasos de calentamiento y registro
        if cfg["reduce_lines_for_testing"]:
            warmup_steps = 1
            logging_steps = 1
        else:
            warmup_steps = int(1000/cfg["batch_size"])
            logging_steps = int((cfg["num_epochs"]*len(train_dataset))/(cfg["batch_size"]*30))
            print(f"logging_steps: {logging_steps}")

        # Definición de argumentos de entrenamiento
        training_args = TrainingArguments(
            output_dir='./results',         
            num_train_epochs=cfg["num_epochs"],             
            per_device_train_batch_size=cfg["batch_size"], 
            per_device_eval_batch_size=cfg["batch_size"], 
            warmup_steps=warmup_steps,                
            weight_decay=0.01,            
            logging_dir='./logs',           
            load_best_model_at_end=False,
            learning_rate=cfg["lr"],
            logging_steps=logging_steps,
            save_strategy='no',
            evaluation_strategy=evaluation_strategy,    
            report_to="none",
            seed=int(datetime.now().timestamp())
        )

    # Entrenamiento del modelo y evaluación

        # Inicialización del entrenador
        trainer = Trainer(
            model=model,                         
            args=training_args,                  
            train_dataset=train_dataset,         
            eval_dataset=eval_datasets,          
            compute_metrics=compute_metrics,     
        )

        # Entrenar el modelo
        trainer.train()

        # Evaluar el modelo en el conjunto de validación si está disponible
        if "dev" in eval_datasets:
            val_predict_results = trainer.predict(
                test_dataset=eval_datasets["dev"],
            )

        # Evaluar el modelo en el conjunto de prueba final si está disponible
        if final_test_dataset is not None:
            test_predict_results = trainer.predict(
                test_dataset=final_test_dataset,
            )
            test_metrics = compute_metrics(test_predict_results)
            for key, val in test_metrics.items():
                print(f"final_test/{key}: {val}")

            # Registrar las salidas de prueba si se especifica
            if cfg["log_test_outputs"]:
                print(f"Logging all test outputs...")
                prediction_results = test_predict_results._asdict()
                with open(os.path.join(cfg["model_save_location"], 'test_outputs.txt'), 'w') as f:
                    f.write("\n".join([str(np.argmax(x)) for x in prediction_results["predictions"]]))

        # Guardar el modelo y el tokenizer si se especifica una ubicación para guardar
        if cfg["model_save_location"] is not None:
            dirname = cfg["model_save_location"]
            if dirname is not None:
                print(f"Saving model to {dirname}")
                if not os.path.exists(dirname):
                    os.makedirs(dirname)
                trainer.save_model(dirname)
                tokenizer.save_pretrained(dirname)

    # Función para inferir con un modelo entrenado
    # Modo de inferencia
    elif cfg["inference_fpath"] is not None:
        # Cargar los datos de inferencia
        with open(cfg["inference_fpath"], "r") as file:
            data = json.load(file)
        if cfg["reduce_lines_for_testing"]:
            data = data[:100]

        results = []
        # Realizar inferencia en lotes
        for i in tqdm(range(0, len(data), cfg["batch_size"]), desc="Inference"):
            batch = data[i:i+cfg["batch_size"]]
            encodings = tokenizer(batch, truncation=True, padding=True, max_length=cfg["max_len"], return_tensors="pt")
            input_ids = to_cuda(encodings['input_ids'])
            attention_mask = to_cuda(encodings['attention_mask'])
            with torch.no_grad():
                logits = model(input_ids, attention_mask=attention_mask).logits

            results += torch.max(logits, dim=-1).indices.cpu().numpy().tolist()

        # Guardar los resultados de la inferencia
        output_fname = f'{os.path.basename(cfg["inference_fpath"]).split(".")[0]}_labels_{os.path.basename(os.path.normpath(cfg["model_name"]))}.json'
        with open(os.path.join(cfg["inference_results_savepath"], output_fname), 'w') as outfile:
            json.dump(results, outfile, indent=3)

    else:
        print("Either train_fpath or inference_fpath needs to be given. Doing nothing...")

In [10]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(prog='Bert finetuning', description='This program finetunes a BERT model')
    parser.add_argument('-c', '--config_path', required=True)
    args = parser.parse_args()

    cfg, _ = parse(args.config_path)
    for c in cfg:
        random.seed(int(datetime.now().timestamp()))
        torch.manual_seed(int(datetime.now().timestamp()))
        np.random.seed(int(datetime.now().timestamp()))
        main(c)

usage: Bert finetuning [-h] -c CONFIG_PATH
Bert finetuning: error: the following arguments are required: -c/--config_path


SystemExit: 2